# KNMI weather data via API

#### Find a how-to guide
The developer portal has a how-to: https://developer.dataplatform.knmi.nl/get-started

#### Pick which catalogue you need
https://dataplatform.knmi.nl/catalog/index.html

I need this one:
Meteo data - daily quality controlled climate data knmi, the netherlands

#### Obtain the API key
Available till 25 May 2021

5e554e19274a9600012a3eb1b626f95624124cf89e9b5d74c3304520

50 requests per minute, 3000 requests per hour

#### KNMI-code
how to list files in dataset:
https://api.dataplatform.knmi.nl/open-data/v1/datasets/{datasetName}/versions/{versionId}/files

how to get download for a file in the dataset:
https://api.dataplatform.knmi.nl/open-data/v1/datasets/{datasetName}/versions/{versionId}/files/{filename}/url


In [83]:
import requests
import json 
import pandas as pd
from pandas import json_normalize
from netCDF4 import Dataset
import numpy as np
import datetime

#### Getting the key in the API - documentation
https://www.digitalocean.com/community/tutorials/getting-started-with-python-requests-get-requests 

In [6]:
API_KEY = '5e554e19274a9600012a3eb1b626f95624124cf89e9b5d74c3304520'
response = requests.get('https://api.dataplatform.knmi.nl/open-data/v1/datasets/etmaalgegevensKNMIstations/versions/1/files', 
                        headers ={'Authorization': API_KEY})

response #[429] too many requests - wait between requests

<Response [200]>

In [9]:
results = response.json()
results

{'maxResults': 10,
 'resultCount': 1,
 'startAfterFilename': '',
 'files': [{'filename': 'KIS___OPER_P___OBS_____L2.nc',
   'size': 3842242,
   'lastModified': '2021-05-14T12:00:45+00:00'}],
 'isTruncated': False}

In [11]:
data = pd.DataFrame(results)
data.head()

,maxResults,resultCount,startAfterFilename,files,isTruncated
0,10,1,,"{'filename': 'KIS___OPER_P___OBS_____L2.nc', '...",False


In [29]:
data["files"][0]

{'filename': 'KIS___OPER_P___OBS_____L2.nc',
 'size': 3842242,
 'lastModified': '2021-05-14T12:00:45+00:00'}

In [3]:
nc_data = Dataset('KIS___OPER_P___OBS_____L2.nc')

<class 'netCDF4._netCDF4.Variable'>
vlen station(station)
    long_name: station name
    cf_role: timeseries_id
vlen data type: <class 'str'>
unlimited dimensions: 
current shape = (50,)

In [4]:
print(nc_data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    featureType: timeSeries
    Conventions: CF-1.4
    title: KIS___OPER_P___OBS_____L2
    institution: Royal Netherlands Meteorological Institute (KNMI)
    source: Royal Netherlands Meteorological Institute (KNMI)
    history: File created from KIS ASCII file. 
    references: http://data.knmi.nl
    comment: none
    dimensions(sizes): station(50), time(25701)
    variables(dimensions): <class 'str'> station(station), float64 time(time), float64 lat(station), float64 lon(station), int16 DDVEC(station, time), int16 FHVEC(station, time), int16 TG(station, time), int16 RH(station, time), int16 UG(station, time), int16 EV24(station, time), int16 PG(station, time), |S1 iso_dataset(), |S1 product(), |S1 projection()
    groups: 


In [6]:
#check out the keys
nc_data.variables.keys()

dict_keys(['station', 'time', 'lat', 'lon', 'DDVEC', 'FHVEC', 'TG', 'RH', 'UG', 'EV24', 'PG', 'iso_dataset', 'product', 'projection'])

In [125]:
#how to get the long name of the dict keys, 
print(nc_data.variables['station'].long_name,  '\t\t\t units: no units attribute')
print(nc_data.variables['time'].long_name, '\t\t units:' , nc_data.variables['time'].units)
print(nc_data.variables['lat'].long_name, '\t\t units:' , nc_data.variables['lat'].units)
print(nc_data.variables['lon'].long_name , '\t\t units:' , nc_data.variables['lon'].units)
print(nc_data.variables['DDVEC'].long_name , '\t units:' , nc_data.variables['DDVEC'].units)
print(nc_data.variables['FHVEC'].long_name , '\t units:' , nc_data.variables['FHVEC'].units)
print(nc_data.variables['TG'].long_name, '\t\t\t units:' , nc_data.variables['TG'].units)
print(nc_data.variables['RH'].long_name, '\t\t units:' , nc_data.variables['RH'].units)
print(nc_data.variables['UG'].long_name, '\t\t units:' , nc_data.variables['UG'].units)
print(nc_data.variables['EV24'].long_name, '\t units:' , nc_data.variables['EV24'].units)
print(nc_data.variables['PG'].long_name, '\t units:' , nc_data.variables['PG'].units)
#print(nc_data.variables['iso_dataset'].long_name)
print(nc_data.variables['product'].long_name)
#print(nc_data.variables['projection'].long_name)

station name 			 units: no units attribute
time of measurement 		 units: seconds since 1950-01-01 00:00:00
station  latitude 		 units: degrees_north
station longitude 		 units: degrees_east
Vector average wind direction 	 units: degree
Vector average wind speed 	 units: m/s
Temperature 			 units: degrees Celsius
Precipitation sum 		 units: mm
Relative humidity 		 units: %
Reference crop evapotranspiration (Makkink) 	 units: mm
Air pressure at sea level 	 units: hPa
ADAGUC Data Products Standard


In [119]:
# select the variables you need (and rename unclear names) 
station = nc_data.variables['station']
time = nc_data.variables['time']
lat = nc_data.variables['lat']
lon = nc_data.variables['lon']
wind_dir = nc_data.variables['DDVEC']
wind_spd = nc_data.variables['FHVEC']
temp = nc_data.variables['TG']
rain = nc_data.variables['RH']

In [120]:
#check out the dimensions of variables
print(temp.dimensions)
print(wind_dir.dimensions)
print(wind_spd.dimensions)
print(rain.dimensions)
print(station.dimensions)
print(lon.dimensions)
print(lat.dimensions)
print(time.dimensions)

('station', 'time')
('station', 'time')
('station', 'time')
('station', 'time')
('station',)
('station',)
('station',)
('time',)


In [128]:
# now save all data in the variable in the variable you created before
# select everything by --> [:]
station = nc_data.variables['station'][:]
time = nc_data.variables['time'][:]
lat = nc_data.variables['lat'][:]
lon = nc_data.variables['lon'][:]
wind_dir = nc_data.variables['DDVEC'][:]
wind_spd = nc_data.variables['FHVEC'][:]
temp = nc_data.variables['TG'][:]
rain = nc_data.variables['RH'][:]

In [134]:
print(station)

#stations are in numbers. On this site are the names belonging to the numbers: https://www.knmi.nl/nederland-nu/klimatologie/daggegevens
# I need stations: # De Bilt Twente Eelde Lauwersoog Lelystad Den Helder Hoek van Holland Vlissingen  Ell Arcen 
#Den Helder is not in the list.. which is weird..

my_stations ={"De Bilt" : 260, "Twente" : 290, "Eelde": 280, "Lauwersoog": 277, "Lelystad": 269, "Den Helder": '?' , "Hoek van Holland": 330, "Vlissingen": 310, "Ell": 377, "Arcen": 391}
my_stations


['391' '370' '331' '315' '324' '375' '380' '240' '286' '310' '283' '280'
 '273' '323' '249' '377' '316' '313' '277' '348' '308' '319' '215' '278'
 '285' '343' '225' '330' '267' '269' '344' '275' '235' '257' '290' '350'
 '251' '210' '248' '279' '258' '356' '209' '265' '312' '340' '260' '270'
 '311' '242']


{'De Bilt': 260,
 'Twente': 290,
 'Eelde': 280,
 'Lauwersoog': 277,
 'Lelystad': 269,
 'Den Helder': '?',
 'Hoek van Holland': 330,
 'Vlissingen': 310,
 'Ell': 377,
 'Arcen': 391}

In [50]:
time

masked_array(data=[3.1536000e+07, 3.1622400e+07, 3.1708800e+07, ...,
                   2.2518432e+09, 2.2519296e+09, 2.2520160e+09],
             mask=False,
       fill_value=1e+20)

### Convert dates to seconds from 1950

In [118]:
# The KNMI dataset contains information about a lot of days. 
# Calculate the time-range you need, based on seconds since 1950-01-01 00:00:00
# The race ends 8 days and 18h later. I'll just take 9 full days as a range

start_2018 = datetime.datetime(2018, 5, 1, 8)
end_2018 = datetime.datetime(2018, 5, 9)

start_2019 = datetime.datetime(2019, 5, 1, 8)
end_2019 = datetime.datetime(2019, 5, 9)

start_2020 = datetime.datetime(2020, 8, 29, 8)
end_2020 = datetime.datetime(2020, 9, 6)

start_2021 = datetime.datetime(2021, 4, 30, 8)
end_2021 = datetime.datetime(2021, 5, 9)

def date_sec(date):
    """
    input: a datetime object
    output: the number of seconds passed between 19501010 00:00:00 and the given datetime
    """
    #seconds in a year: 365 days * 24h * 60min * 60 sec = 31536000 seconds.
    #seconds in a 'schrikkel'year = 366 * 24 * 60 * 60 = 31622400 seconds.
    
    knmi_start = datetime.datetime(1950, 1, 1)
    passed_seconds = (date - knmi_start).total_seconds()
    return (passed_seconds)



print("Range of seconds between first and last day of 2018:", date_sec(start_2018), 'till', date_sec(end_2018))
print("Start date of 2021 in seconds since 1950: ", date_sec(start_2021))


Range of seconds between first and last day of 2018: 2156313600.0 till 2156976000.0
Start date of 2021 in seconds since 1950:  2250921600.0


In [ ]:
# example code 
# tutorial: https://www.youtube.com/watch?v=hrm5RmsVXo0

# lat = data.variables['lat'][:]
# lon = data.variables['lon'][:]

# min_index_lat --> calculates the lat-value you want to use
# min_index_lon --> calculates the lon-value you want to use

# temp = data.variables["temp"]

# #create dataframe
# starting_date  = something
# ending_date = something else

# date_range = pd.date_range(start = starting_date, end = ending_date)

# # create dataframe with the length pf the date_range filled with  0's
# df = pd.DataFrame(0, columns = ["temp"], index = date_range)

# #creates a series with the size of the temp values
# dt = np.arange(0, data.variables['temp'].size) 

# #puts the temperatures that belong to the right lat&lon in the df
# for i in dt:
#     df.iloc[i] = temp[i, min_index_lat, min_index_lon]
# df    
    

In [81]:
nc_data.variables['time']

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: time of measurement
    standard_name: time
    units: seconds since 1950-01-01 00:00:00
unlimited dimensions: 
current shape = (25701,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [ ]:
#alternative way: give all data for this variable

time_data = nc_data.variables['time'][:]
time_data

In [40]:
temp_data = nc_data.variables['TG'][:]
# temp_data
#or use (result is the same):
temp_dt = temp[:]
# temp_dt

variables(dimensions): 
<class 'str'> station(station), 
float64 time(time), 
float64 lat(station), 
float64 lon(station), 
int16 DDVEC(station, time), 
int16 FHVEC(station, time), 
int16 TG(station, time), 
int16 RH(station, time), 
int16 UG(station, time), 
int16 EV24(station, time), 
int16 PG(station, time), 
|S1 iso_dataset(), 
|S1 product(),
|S1 projection()

In [ ]:
nc_data.variables['station']

PYCURL https://stackabuse.com/using-curl-in-python-with-pycurl/
on curl: https://medium.com/@MissAmaraKay/what-is-curl-and-why-is-it-all-over-api-docs-b141c33805e0
python requests: https://www.digitalocean.com/community/tutorials/getting-started-with-python-requests-get-requests
KNMI API als package: https://pypi.org/project/knmi-py/
KNMI example scripts: https://developer.dataplatform.knmi.nl/example-scripts#list-10-current-files-get-first